In [75]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

Считайте датасет в Python (можете сразу грузить все в память, выборка небольшая), выясните, что используется в качестве разделителей и как проставляются метки классов.

Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".

In [35]:
filename = 'smsspamcollection/SMSSpamCollection.txt'



with codecs.open(filename, "r", "utf_8_sig" ) as file:
    text = file.readlines()

y = []
X = []
for line in text:
    line_target, line_features = line.split('\t')
    X.append(line_features.strip('\n'))
    y.append(line_target)
    
n_uniques = {}
for label in y:
    count = n_uniques.get(label, 0)
    n_uniques[label] = count + 1
    
print("Ham count: {} \nSpam count: {}".format(n_uniques['ham'], n_uniques['spam']))

y = list(map(lambda x: 1 if x == 'spam' else 0, y))

Ham count: 4827 
Spam count: 747


Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получите из списка текстов матрицу признаков X.

Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установите random_state=2. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.

In [53]:
vectorizer = CountVectorizer()
X_vec = vectorizer.fit_transform(X).toarray()

log_model = LogisticRegression(random_state=2)
score = cross_val_score(log_model, X_vec, y, cv=10, scoring="f1").mean()

print('Cross validation score: {}'.format(score))

In [ ]:
with open('answer1.txt', 'w') as file:
    file.write(str(round(score, 1)))

А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

"FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"

"FreeMsg: Txt: claim your reward of 3 hours talk time"

"Have you visited the last lecture on physics?"

"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"

"Only 99$"

Прогнозы классификатора (0 - не спам, 1 - спам), записанные через пробел, будут ответом в одном из вопросов ниже.

In [67]:
log_model = LogisticRegression(random_state=42)
log_model.fit(X_vec, y)

X_test = ["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
          "FreeMsg: Txt: claim your reward of 3 hours talk time",
          "Have you visited the last lecture on physics?",
          "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
          "Only 99$"]

X_test_vec = vectorizer.transform(X_test).toarray()
predictions = log_model.predict(X_test_vec)

print('Classificator predictions: {}'.predict(predictions))

In [69]:
with open('answer2.txt', 'w') as file:
    file.write(' '.join(str(pred) for pred in predictions))

Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.


In [70]:
%%time
vectorizers = [CountVectorizer(ngram_range=(2,2)),
               CountVectorizer(ngram_range=(3,3)),
               CountVectorizer(ngram_range=(1,3))]

model = LogisticRegression()
scores = []
for vec in vectorizers:
    X_vec_ngram = vec.fit_transform(X).toarray()
    scores.append(cross_val_score(model, X_vec_ngram, y, cv=10, scoring='f1').mean())

Wall time: 9min 4s


In [72]:
with open('answer3.txt', 'w') as file:
    file.write(' '.join(str(round(score, 2)) for score in scores))

Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

По какой-то причине обучение наивного байесовского классификатора через Pipeline происходит с ошибкой. Чтобы получить правильный ответ, отдельно посчитайте частоты слов и обучите классификатор.

In [74]:
%%time
vectorizers = [CountVectorizer(ngram_range=(2,2)),
               CountVectorizer(ngram_range=(3,3)),
               CountVectorizer(ngram_range=(1,3))]

model = MultinomialNB()
scores = []
for vec in vectorizers:
    X_vec_ngram = vec.fit_transform(X).toarray()
    scores.append(cross_val_score(model, X_vec_ngram, y, cv=10, scoring='f1').mean())

Wall time: 5min 15s


In [76]:
with open('answer4.txt', 'w') as file:
    file.write(' '.join(str(round(score, 2)) for score in scores))

Попробуйте использовать в логистической регрессии в качестве признаков Tf*idf из TfidfVectorizer на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к tf*idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [77]:
vectorizer = TfidfVectorizer()
X_vec = vectorizer.fit_transform(X).toarray()

log_model = LogisticRegression(random_state=2)
score = cross_val_score(log_model, X_vec, y, cv=10, scoring="f1").mean()

print('Cross validation score: {}'.format(score))

Cross validation score: 0.8528304419417478


In [ ]:
with open('answer5.txt', 'w') as file:
    file.write(str(round(score, 1)))